# Data Preprocessing steps

V1: https://mconverter.eu/convert/docx/md/
V2: https://word2md.com/

1. First, substitute "---" with "-" in the original file - v1
2. Run the following code

In [2]:
from src.utils.strip_markdown import strip_markdown

path = "../data/grammar-level-1/final/grammar_list_word2md.md"
clean_path = "../data/grammar-level-1/final/grammar_list_clean_word2md.md"

with open(path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# find all indices of description lines
desc_idxs = [i for i, L in enumerate(lines) if 'Описание' in L]
rev_idxs = list(reversed(desc_idxs))

first = True
for i, idx in enumerate(rev_idxs):
    hdr_idx = idx - 2

    # format header line as H2
    original = lines[hdr_idx].rstrip('\n')
    original = strip_markdown(original)
    lines[hdr_idx] = f'## {original}\n'

    # insert delimiter above it unless it's the first line
    if i < len(rev_idxs) - 1:
        lines.insert(hdr_idx, '\n---\n')

with open(clean_path, 'w', encoding='utf-8') as f:
    f.writelines(lines)

In [3]:
import re

with open("../data/grammar-level-1/final/grammar_list_clean_word2md.md", 'r', encoding='utf-8') as f:
    text = f.read()

cleaned = text
cleaned = cleaned.replace('## ###', '##')
cleaned = re.sub(r'\\([^\w\s])', r'\1', cleaned)
cleaned = cleaned.replace('—', '-')
cleaned = cleaned.replace('\t-', '-')
cleaned = cleaned.replace('<br/>', '\n')
cleaned = cleaned.replace('  \n', '\n')
cleaned = re.sub(r'\[EMSP\]', '\t', cleaned)
cleaned = re.sub(r'"(.*?)"', r'«\1»', cleaned)
cleaned = re.sub(r"'(.*?)'", r'«\1»', cleaned)
cleaned = re.sub(r'\*\*([^\s\uAC00-\uD7AF])\*\*', r'\1', cleaned)

with open("../data/grammar-level-1/final/grammar_list_clean_word2md.md", 'w', encoding='utf-8') as f:
    f.write(cleaned)

3. "## ###" на "##" BOTH
4. Use this regex: \{([^}]*)\} to - v1
    - Delete duplicating titles
    - Empty titles (will be fixed)
    - Delete any {mark}s
5. "** **" на " " - вроде даже не обязательно - v1
6. "ENTER >" на "ENTER \t" - v1
7. Signs with \ in the beginning: \", \~, \|, \., \*, \', \[, \], \+ BOTH
8. "--" на "-" - v1
9. "—" на "-" - v2
10. "\t-" на "-" - v2
11. "[EMSP]" на "\t" BOTH
12. Fix Long Grammar names accidentally separated. Use Obsidian table of content - easier to spot - v1
13. Use this regex \*\*([^\s\uAC00-\uD7AF])\*\* to spot weird bolding of single characters (e.g. **И**спользуется с существительными и выражает значение **«толь...) - V2
14. Add | separator between Korean and Russian grammar names BOTH
15. "  ENTER" (space+space+ENTER) на "ENTER"


16. "•" на "-"


## Чекнуть \~기(가) | ~기(가) + прилагательное!

In [4]:
LEVEL = 1

def parse_entry(text):
    """
    Parse a single grammar explanation into a dictionary.
    - Separates Korean and Russian grammar names from the header.
    - Adds a predefined level.
    - Extracts "Смежные темы" into a 'related_grammar' list.
    - Stores the remaining description in the 'content' field.
    """
    lines = text.strip().splitlines()

    # --- 1. Parse Header ---
    header_line = lines[0].strip()

    # Clean the markdown: "## **이/가 именительный падеж**" -> "이/가 именительный падеж" -> 이/가 | именительный падеж
    clean_header = ""
    if header_line.startswith('##'):
        clean_header = header_line.replace("*", "").replace("#", "").strip()
        if '|' not in clean_header:
            m = re.match(r'\s*([^\u0400-\u04FF]+?)\s*([«\u0400-\u04FF].+)', clean_header)
            if m:
                kor, rus = m.group(1).strip(), m.group(2).strip()
                clean_header = f"{kor}|{rus}"

    parts = clean_header.split('|', 1)
    grammar_name_kr = parts[0].strip()
    grammar_name_rus = parts[1].strip() if len(parts) > 1 else ""

    # --- 2. Parse Content and Related Topics ---
    content_lines = lines[1:]
    related_grammar = []

    # Trim any trailing empty lines to easily access the last content line
    while content_lines and not content_lines[-1].strip():
        content_lines.pop()

    # Check if the last non-empty line contains the related topics
    if content_lines and content_lines[-1].strip().startswith('Смежные темы:'):
        # Extract the line and remove it from the content_lines list
        related_topics_line = content_lines.pop().strip()

        # Get the string part after the colon
        topics_str = related_topics_line.split(':', 1)[1]

        # Split the topics by comma, strip whitespace from each, and filter out any empty results
        related_grammar = [topic.strip() for topic in topics_str.split(',') if topic.strip()]

    # Trim any leading empty lines from the remaining content
    while content_lines and not content_lines[0].strip():
        content_lines.pop(0)

    # Join the remaining lines to form the main content
    content = "\n".join(content_lines)

    return {
        "grammar_name_kr": grammar_name_kr,
        "grammar_name_rus": grammar_name_rus,
        "level": LEVEL,
        "related_grammars": related_grammar,
        "content": content
    }

In [5]:
def parse_input_md(input_text):
    parts = input_text.split('---')
    entries = []
    for part in parts:
        part = part.strip()
        if not part:
            continue
        entry = parse_entry(part)
        entries.append(entry)
    return entries

with open("../data/grammar-level-1/final/grammar_list_clean_word2md.md", "r", encoding="utf-8") as infile:
    grammar_text = infile.read()

In [6]:
grammar_list = parse_input_md(grammar_text)

In [7]:
len(grammar_list)

108

In [9]:
grammar_list[69]

{'grammar_name_kr': '(이)나',
 'grammar_name_rus': '«даже», «аж»',
 'level': 1,
 'related_grammars': ['**(이)나 «или»', '(이)나 «что угодно»**'],
 'content': '**Описание:**\nЧастица (이)나 может использоваться для подчеркивания того, что количество, масштаб или степень чего-то **больше ожидаемого**.\n\n**Форма:**\n**Существительное + (이)나**\n> Если существительное заканчивается на **согласную** → добавляется **이나**\n> Если на **гласную** → добавляется **나**\n\n**Примеры:**\n혼자 피자 한 판**이나** 먹었어요.\nСъел аж целую пиццу один.\n\n하루에 커피를 다섯 잔**이나** 마셔요.\nВыпиваю аж пять чашек кофе в день.\n\n그 배우는 영화에 열 편**이나** 나왔어요.\nЭтот актёр снялся аж в десяти фильмах.\n\n시험을 세 번**이나** 봤어요.\nСдавала экзамен целых три раза.\n\n**Примечания:**\n1. В этом значении частица часто ставится после числительных или существительных с количеством.\n'}

In [10]:
import pandas as pd
import hashlib

df = pd.DataFrame(grammar_list)

def hash_text(text: str) -> str:
    return hashlib.sha256(text.encode('utf-8')).hexdigest()[:32]

grammar_name = df["grammar_name_kr"] + " " + df["grammar_name_rus"]
df.index = grammar_name.apply(hash_text)
df.index.name = "grammar_id"
df

,grammar_name_kr,grammar_name_rus,level,related_grammars,content
grammar_id,,,,,
602036e488ddf85722f218e49087c45a,이/가,именительный падеж,1,"[은/는 выделительная частица, 께서]",**Описание:**\nЧастицы **이/가** обозначают **им...
bd5253ce8e0b11a7ee24bce4850cb389,와/과,"«и», перечисление существительных",1,"[하고, (이)랑]",**Описание:**\nИспользуется для перечисления п...
27bcb1cd7ba042c6f7214fdd1ea1cf6c,와/과,"«с», совместное действие",1,"[하고, (이)랑]","**Описание:**\nУказывает на лицо или объект, с..."
2749970a6a7f10f83726d189e561fe78,까지,«до»,1,"[부터, 에서 «из»]",**Описание:**\nЧастица **까지** используется для...
c7c87e5469bf9ac9161ae90d5bf3e702,께서,именительный падеж (вежл.),1,"[이/가, 께, 께서는, -(으)시-, -(으)세요]",**Описание:**\nЭто **вежливая форма** именител...
...,...,...,...,...,...
f2fc97bee878b07d1f8280fbf6ca5543,V + -아/어도 되다,"«можно, разрешается»",1,[],**Описание:**\nКонструкция -아/어도 되다 использует...
e652e703e0b0a78afd83dd4f2d693276,V + -지 말다,«не делайте»,1,[-(으)세요],**Описание:**\nКонструкция -지 말다 используется ...
4c037898975a6fdecdcd870c385dd03f,V + -(으)ㄹ 수밖에 없다,"«ничего не остаётся, кроме как…»",1,[],**Описание:**\nКонструкция -(으)ㄹ 수밖에 없다 исполь...


In [11]:
type(df.iloc[0]["related_grammars"])

list

In [12]:
df[df.content.apply(lambda x: x.startswith('**Описание'))].info()

<class 'pandas.core.frame.DataFrame'>
Index: 108 entries, 602036e488ddf85722f218e49087c45a to 8d6f9dc436fd95966cdf414cffd6377a
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   grammar_name_kr   108 non-null    object
 1   grammar_name_rus  108 non-null    object
 2   level             108 non-null    int64 
 3   related_grammars  108 non-null    object
 4   content           108 non-null    object
dtypes: int64(1), object(4)
memory usage: 9.1+ KB


## Add converting СМЕЖНЫЕ ГРАММАТИКИ to IDs

In [13]:
df.to_pickle("../data/grammar-level-1/final/grammar_list_clean_word2md.pkl")
new_df = pd.read_pickle("../data/grammar-level-1/final/grammar_list_clean_word2md.pkl")

In [14]:
type(df.iloc[0]["related_grammars"])

list

In [15]:
type(new_df.iloc[0]["related_grammars"])

list

In [16]:
new_df

,grammar_name_kr,grammar_name_rus,level,related_grammars,content
grammar_id,,,,,
602036e488ddf85722f218e49087c45a,이/가,именительный падеж,1,"[은/는 выделительная частица, 께서]",**Описание:**\nЧастицы **이/가** обозначают **им...
bd5253ce8e0b11a7ee24bce4850cb389,와/과,"«и», перечисление существительных",1,"[하고, (이)랑]",**Описание:**\nИспользуется для перечисления п...
27bcb1cd7ba042c6f7214fdd1ea1cf6c,와/과,"«с», совместное действие",1,"[하고, (이)랑]","**Описание:**\nУказывает на лицо или объект, с..."
2749970a6a7f10f83726d189e561fe78,까지,«до»,1,"[부터, 에서 «из»]",**Описание:**\nЧастица **까지** используется для...
c7c87e5469bf9ac9161ae90d5bf3e702,께서,именительный падеж (вежл.),1,"[이/가, 께, 께서는, -(으)시-, -(으)세요]",**Описание:**\nЭто **вежливая форма** именител...
...,...,...,...,...,...
f2fc97bee878b07d1f8280fbf6ca5543,V + -아/어도 되다,"«можно, разрешается»",1,[],**Описание:**\nКонструкция -아/어도 되다 использует...
e652e703e0b0a78afd83dd4f2d693276,V + -지 말다,«не делайте»,1,[-(으)세요],**Описание:**\nКонструкция -지 말다 используется ...
4c037898975a6fdecdcd870c385dd03f,V + -(으)ㄹ 수밖에 없다,"«ничего не остаётся, кроме как…»",1,[],**Описание:**\nКонструкция -(으)ㄹ 수밖에 없다 исполь...
